In [1]:
%%time
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , precision_recall_curve , f1_score
from sklearn.metrics import classification_report , roc_auc_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from joblib import dump, load
import joblib

Wall time: 18 s


In [2]:
rfc = joblib.load('modelmmww.joblib')

In [3]:
df = pd.read_csv('RosterLinkedIn_YaleM2.csv')
#df2 = pd.read_csv('RosterLinkedIn_CornellW.csv')

In [4]:
#df['SameTennis_yn'] = (df['SameName_yn'] + df['tennis_yn'])
#df2['SameTennis_yn'] = (df2['SameName_yn'] + df2['tennis_yn'])


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 34 columns):
FirstLast         425 non-null object
Criteria          425 non-null object
FirstLast.1       425 non-null object
Year              425 non-null float64
YearsPlayed       425 non-null float64
RosterLink        40 non-null object
Hometown          40 non-null object
HS                37 non-null object
ClassYear         40 non-null object
College           425 non-null object
First             425 non-null object
Gender            425 non-null int64
Last              425 non-null object
MidInit           0 non-null float64
MidName           0 non-null float64
Name              40 non-null object
RosterYear        40 non-null object
LName             363 non-null object
SearchResult      363 non-null object
Headline          363 non-null object
Location          363 non-null object
Company           351 non-null object
School            363 non-null object
Activites         363 non-nu

In [6]:
#This was for CornellW_merge which had names with no result, e.g. Nan rows
df.dropna(subset = ['SearchResult'], inplace=True)
df.set_index('FirstLast', inplace=True, drop=False)
df = df.select_dtypes(exclude=['object'])
#df_label = df[['EC_Review']]
#df.drop(['EC_Review', 'prediction', 'Decade', 'YearsPlayed'], axis=1, inplace=True)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.drop(['MidInit', 'MidName'], axis=1, inplace=True)

In [9]:
df

,Year,YearsPlayed,Gender,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn,Decade
FirstLast,,,,,,,,,,,,,
Victor KIAM,"1,947.00",1.00,1,22.00,27.00,22.00,22.00,22.00,0.00,0.00,0.00,0.00,"1,940.00"
Charles RITCHIE,"1,947.00",2.00,1,32.00,33.00,45.00,77.00,77.00,0.00,0.00,0.00,0.00,"1,940.00"
James BOYLE,"1,947.00",1.00,1,64.00,64.00,100.00,100.00,100.00,0.00,0.00,0.00,0.00,"1,940.00"
Henry EWALD,"1,947.00",1.00,1,15.00,18.00,46.00,46.00,54.00,0.00,0.00,0.00,0.00,"1,940.00"
Robert SUTTON,"1,947.00",1.00,1,43.00,50.00,87.00,87.00,87.00,0.00,0.00,0.00,0.00,"1,940.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dylan King,"2,020.00",4.00,1,78.00,88.00,82.00,83.00,82.00,1.00,0.00,0.00,1.00,"2,020.00"
Ryan Cheng,"2,020.00",4.00,1,63.00,67.00,63.00,63.00,63.00,0.00,0.00,0.00,0.00,"2,020.00"
Michael Sun,"2,020.00",2.00,1,86.00,90.00,90.00,90.00,90.00,0.00,0.00,0.00,0.00,"2,020.00"


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, Victor KIAM to Alan Sou
Data columns (total 14 columns):
Year              353 non-null float64
YearsPlayed       353 non-null float64
Gender            353 non-null int64
fuzz_ratio        353 non-null float64
fuzz_pratio       353 non-null float64
fuzz_tokensort    353 non-null float64
fuzz_tokenset     353 non-null float64
fuzz_wratio       353 non-null float64
tennis_yn         353 non-null float64
School_yn         353 non-null float64
SameName_yn       353 non-null float64
SameTennis_yn     353 non-null float64
Decade            353 non-null float64
prediction        353 non-null float64
dtypes: float64(13), int64(1)
memory usage: 41.4+ KB


In [11]:
X = df.values

In [12]:
X.shape

(353, 13)

In [13]:
rfc

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [14]:
rfcpredictX = rfc.predict(X)

In [15]:
rfcpredictX

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0.

In [16]:
df['prediction'] = rfcpredictX

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, Victor KIAM to Alan Sou
Data columns (total 14 columns):
Year              353 non-null float64
YearsPlayed       353 non-null float64
Gender            353 non-null int64
fuzz_ratio        353 non-null float64
fuzz_pratio       353 non-null float64
fuzz_tokensort    353 non-null float64
fuzz_tokenset     353 non-null float64
fuzz_wratio       353 non-null float64
tennis_yn         353 non-null float64
School_yn         353 non-null float64
SameName_yn       353 non-null float64
SameTennis_yn     353 non-null float64
Decade            353 non-null float64
prediction        353 non-null float64
dtypes: float64(13), int64(1)
memory usage: 41.4+ KB


In [24]:
df.to_csv('Predict_YaleM2_mmww.csv', index=True)